# 📅 Day 2 — Data Cleaning & Preparation

*RTU Data Analysis & Visualization CPD course*

**📚 Instruction (3h)**  
- 🧹 Handling missing values  
- 🗑 Removing duplicates  
- 🔄 Data type conversion  
- 📅 Parsing dates  
- 🏗 Feature engineering basics  
- 🔗 Combining datasets  
- 🏷 Intro to categorical encoding  

**🛠 Practical (1h)**  
- 🧽 Clean a messy dataset  
- 🔀 Merge with a secondary dataset  

**🔄 Reflection (1h)**  
- 🧐 Review: common pitfalls in cleaning  
- 💬 Discuss real-world cleaning challenges  
- 📝 Recap exercise: identify cleaning steps for a small example dataset

## 🎯 Goals for the Day
- Strengthen Python basics (functions, loops, if/else, file handling)
- Learn to process raw messy text files into usable form
- Apply pandas methods to clean incomplete/messy data
- Merge multiple datasets into a single unified dataframe

## 💡 Motivation / Explanation

### Introduction to Data Cleaning and Preparation

- **Why cleaning is critical before analysis**
  - Raw data is almost never ready for direct analysis
  - Errors, inconsistencies, and missing information can distort results
  - Proper cleaning ensures reliability, reproducibility, and trust in analysis outcomes

- **Real-world examples of messy data**
  - 🧹 Handling missing values - Weather records with missing timestamps or corrupt values
  - 🗳️ Survey responses with inconsistent categories (e.g., "Male", "male", "M")
  - 🗑 Removing duplicates - Financial transactions with duplicate entries
  - 🗑 Log files with noise lines, system messages, or broken encodings
  - 📅 Parsing dates - Event logs with inconsistent timestamp formats
  - 🔄 Data type conversion - User age recorded as text instead of numbers

> Think of data cleaning as *“washing vegetables before cooking”* — not exciting, but essential for a good meal.


## Part 1: Python Fundamentals for Data Cleaning

In [ ]:
from pathlib import Path
import sys, platform, os, io, shutil, zipfile, re
from datetime import datetime
import pandas as pd
try:
    import requests
except ImportError:
    requests = None
    print('requests not installed. Install with `%pip install requests`.')

print('pandas:', pd.__version__)
print('Python :', sys.version.split()[0])
print('Runtime:', platform.platform())
print('Now    :', datetime.now().isoformat(timespec='seconds'))

### 🧑‍💻 Functions

In [ ]:
def greet(name: str = 'student') -> str:
    return f"Hello, {name}!"
print(greet())

In [ ]:
def download_and_unzip(url: str, target_folder: str | Path = 'sample_data') -> Path:
    target = Path(target_folder)
    target.mkdir(parents=True, exist_ok=True)
    filename = url.split('/')[-1]
    zip_path = target / filename
    if requests is None:
        raise RuntimeError('requests required.')
    with requests.get(url, stream=True, timeout=60) as r:
        r.raise_for_status()
        with open(zip_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    with zipfile.ZipFile(zip_path, 'r') as zf:
        zf.extractall(target)
    return target


**Practice dataset for Part 1:** `latvia_meteo_1925_messy.zip` (5 text files)

- URL: https://github.com/ValRCS/RTU_Data_Analysis_Visualization_CPD/raw/refs/heads/main/data/latvia_meteo_1925_messy.zip

### 📂 File Handling

In [ ]:
def iter_lines(path: Path):
    with path.open('r', encoding='utf-8', errors='replace') as f:
        for line in f:
            yield line.rstrip('\n')

### 🔄 For Loops

In [ ]:
def count_lines(path: Path) -> tuple[int, int]:
    total, nonempty = 0, 0
    for line in iter_lines(path):
        total += 1
        if line.strip():
            nonempty += 1
    return total, nonempty

### 🌳 If / Else Branching

In [ ]:
def is_good_line(line: str) -> bool:
    s = line.strip()
    if not s: return False
    if s.startswith('#'): return False
    if len(s) < 5: return False
    return True

### 🧹 Building a Cleaning Function

In [ ]:
def clean_file(path: Path) -> tuple[int, int]:
    out_good = path.with_suffix('.good.txt')
    out_bad = path.with_suffix('.bad.txt')
    good, bad = 0, 0
    with path.open('r', encoding='utf-8', errors='replace') as fin, \
         out_good.open('w') as fg, out_bad.open('w') as fb:
        for line in fin:
            if is_good_line(line):
                fg.write(line)
                good += 1
            else:
                fb.write(line)
                bad += 1
    return good, bad

### 📁 Extending to Folders

In [ ]:
def clean_files(folder: Path):
    results = {}
    for file in folder.glob('*.txt'):
        results[file] = clean_file(file)
    return results

### 📥 Loading Cleaned Files into DataFrames

In [ ]:
def load_cleaned_file(path: Path) -> pd.DataFrame:
    with path.open('r') as f:
        lines = [l.strip().split() for l in f if l.strip()]
    maxlen = max(len(r) for r in lines) if lines else 0
    cols = [f'col{i+1}' for i in range(maxlen)] if maxlen else []
    return pd.DataFrame(lines, columns=cols)

## Part 2: Guided Exercise — Latvia Weather Data (Extra Messy)

**Duration:** ~30 minutes  
**Dataset:** `latvia_meteo_1925_extra_messy.zip`  
**URL:** https://github.com/ValRCS/RTU_Data_Analysis_Visualization_CPD/raw/refs/heads/main/data/latvia_meteo_1925_extra_messy.zip

### 🎯 Objective
Convert multiple extra-messy weather text files into **one cleaned file per source**, then load each into a **separate DataFrame**.

### ✅ Success Criteria
- Each original text file has a corresponding `.good.txt` output.
- Each `.good.txt` loads into a DataFrame without errors.
- Basic column consistency achieved (same number of columns and sensible types where possible).

### 🔍 What to Watch For
- Junk header/footer lines (e.g., comments, separators)
- Inconsistent separators (`,`, `;`, tabs, or spaces)
- Missing fields and short/empty lines
- Non-UTF8 characters — use `errors='replace'` if needed

### 🧭 Suggested Workflow
1) **Download & unzip** to `data/` using `download_and_unzip`  
2) **List files** and do a quick **line count** with `count_lines`  
3) **Clean** with `clean_files(data_dir)`  
4) **Load** each cleaned file with `load_cleaned_file`  
5) **Sanity-check**: `.head()`, `.info()`, and simple value counts on key columns

### 🧩 Hints
- If a file still fails to parse, adjust `is_good_line` (e.g., skip lines that start with specific tokens).
- If different files use different separators, handle at **pandas** stage later (Part 3) by re-parsing columns.
- Keep outputs organized: write cleaned files into a `data/cleaned/` subfolder if you choose to extend `clean_files`.

In [ ]:
# --- SKELETON (students fill in) ---
EXTRA_URL = 'https://github.com/ValRCS/RTU_Data_Analysis_Visualization_CPD/raw/refs/heads/main/data/latvia_meteo_1925_extra_messy.zip'
DATA_DIR = Path('data')

# 1) Download & unzip
# download_and_unzip(EXTRA_URL, DATA_DIR)

# 2) Inspect: list files & counts
# for p in sorted(DATA_DIR.glob('*.txt')):
#     print(p.name, '->', count_lines(p))

# 3) Clean all files
# results = clean_files(DATA_DIR)
# results

# 4) Load cleaned files
# dfs_extra = {}
# for p in sorted(DATA_DIR.glob('*.good.txt')):
#     dfs_extra[p.stem] = load_cleaned_file(p)
# {k: v.head() for k, v in dfs_extra.items()}

### 🧪 Checkpoints
- At least **N≥3** cleaned files successfully load into DataFrames.
- No parsing exceptions on `.head()` or `.info()`.
- You can explain (in comments) which rules your `is_good_line` used.

### 🛠 Extension (Optional)
- Write a variant `clean_files(folder, out_dir=Path('data/cleaned'))` that writes outputs into a subfolder.
- Add a **regex-based** `is_good_line_regex` that only keeps lines starting with `YYYY-MM-DD`.

## Part 3: Pandas-Specific Data Cleaning

### Overview
In this section, you will standardize each DataFrame from Part 2 so they share a **common schema** and are ready to merge.

### Target Schema (example)
- `date` (datetime)
- `station` (string/category)
- `t_min` (float)
- `t_max` (float)
- `precip` (float)

### Typical Operations
1. **Column detection & renaming** – bring different column names to a shared set
2. **Type coercion** – numbers via `pd.to_numeric(errors='coerce')`, dates via `pd.to_datetime(errors='coerce')`
3. **Missing values** – `dropna` or `fillna` depending on context
4. **Duplicates** – `.duplicated()` + `.drop_duplicates()`
5. **Categoricals** – normalize text (`strip`, `title`, `upper`) and `astype('category')` if useful
6. **Validation** – quick assertions (e.g., date not null, temperature ranges plausible)

### Step-by-Step Guide
1) **Pick one DataFrame** from `dfs_extra` and print `.head()`, `.columns`, `.info()`
2) **Map columns** to target names (e.g., `temp_min` → `t_min`)
3) **Coerce**:
   - `date = pd.to_datetime(df['date'], errors='coerce')`
   - `df[['t_min','t_max','precip']] = df[['t_min','t_max','precip']].apply(pd.to_numeric, errors='coerce')`
4) **Handle missing**: start conservative (e.g., drop rows missing `date` or all temperature columns)
5) **Standardize station names**: `df['station'] = df['station'].astype(str).str.strip().str.title()`
6) **Check duplicates** and remove
7) **Repeat** for all DataFrames

### Common Pitfalls & Tips
- Treat ambiguous `-` or `NA` strings as missing (`na_values=["-","NA","N/A"]` if you re-read with `read_csv`)
- Some files might have **merged columns**; split using `.str.split(',', expand=True)` when necessary
- If a file lacks a column, create it with `pd.NA` so the schema lines up later

### 🧱 Skeleton: Inspect & Rename

In [ ]:
# Example skeleton for one dataframe named df
# df = dfs_extra['some_file']
# print(df.head()); print(df.columns); df.info()

# rename_map = {
#     'Date': 'date', 'DATE':'date',
#     'Station':'station', 'City':'station',
#     'Tmin':'t_min', 'TminC':'t_min', 'Min':'t_min',
#     'Tmax':'t_max', 'TmaxC':'t_max', 'Max':'t_max',
#     'Precip':'precip', 'Rain':'precip'
# }
# df = df.rename(columns=lambda c: rename_map.get(str(c), str(c).strip().lower()))


### 🧱 Skeleton: Type Coercion & Missing Handling

In [ ]:
# required_cols = ['date','station','t_min','t_max','precip']
# for c in required_cols:
#     if c not in df.columns:
#         df[c] = pd.NA

# df['date'] = pd.to_datetime(df['date'], errors='coerce')
# for c in ['t_min','t_max','precip']:
#     df[c] = pd.to_numeric(df[c], errors='coerce')

# # Drop rows with no usable date
# df = df.dropna(subset=['date'])

# # Optional: fill precip missing with 0 if domain-appropriate
# # df['precip'] = df['precip'].fillna(0)

### 🧱 Skeleton: Text Normalization & Duplicates

In [ ]:
# df['station'] = df['station'].astype(str).str.strip().str.title()
# before = len(df)
# df = df.drop_duplicates()
# print('Removed', before - len(df), 'duplicate rows')

### 🧪 Suggested Sanity Checks

In [ ]:
# assert df['date'].notna().all(), 'Null dates remain'
# # Optional plausibility checks (adjust to real units)
# assert (df['t_min'] <= df['t_max']).dropna().all(), 'Found t_min > t_max'

## Part 4: Merging Cleaned DataFrames

### Goal
Combine all standardized DataFrames into **one big DataFrame** with a **unified column structure**.

### Strategy
1. **Define the target schema** used in Part 3.
2. **Align each DataFrame** to the schema (add missing columns, reorder).
3. **Concatenate** with `pd.concat`.
4. **Final cleanup**: deduplicate, reindex, and sort by date/station.
5. **Save outputs** (`CSV` or `Parquet`) for Day 3 (EDA).

### Integration Checklist
- All DataFrames have columns: `date, station, t_min, t_max, precip`
- Dtypes are consistent across DataFrames
- No catastrophic loss of rows during coercion
- Final row count equals the sum of inputs minus duplicates

### 🧱 Skeleton: Alignment & Concatenation

In [ ]:
# Suppose you have a dict of cleaned dfs: dfs_clean
# target_cols = ['date','station','t_min','t_max','precip']

# def coerce_to_schema(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
#     for c in cols:
#         if c not in df.columns:
#             df[c] = pd.NA
#     # Reorder and drop extras for now
#     return df[cols]

# aligned = [coerce_to_schema(d.copy(), target_cols) for d in dfs_clean.values()]
# big = pd.concat(aligned, axis=0, ignore_index=True)
# big = big.drop_duplicates().reset_index(drop=True)
# big = big.sort_values(['date','station'])
# big.head()

### 🧾 Export for Day 3

In [ ]:
# out_dir = Path('outputs'); out_dir.mkdir(exist_ok=True)
# big.to_csv(out_dir / 'latvia_meteo_1925_cleaned_merged.csv', index=False)
# # Optional: Parquet for speed/size
# # big.to_parquet(out_dir / 'latvia_meteo_1925_cleaned_merged.parquet', index=False)

## 🔄 Reflection
- What kinds of messiness were easier to fix with **Python basics**?
- What kinds of messiness required **pandas**?
- What are the risks of “over-cleaning” or discarding too much data?